In [ ]:
%matplotlib inline

In [ ]:
from __future__ import print_function
import numpy as np
import pylab as plt
import xraylib
import numba

In [ ]:
@numba.jit
def create_phantom_tooth(sx, energy, elem1, elem2, pixel_size, isimplant):
    """
    Create phantom, share is two flowers (with 3 and 6 petals)

    :param sx: size of phantom
    :param energy: energy, set in keV
    :param elem1: Number of the chemical element
    :param elem2: Number of the chemical element
    :param pixel_size: size of one pixel, set in microns
    :return: 2d array of phantom
    """
    xraylib.XRayInit()
    phantom = np.zeros((sx, sx))
    sx_half = sx / 2
    sq = sx / 14
    
    density_1 = xraylib.ElementDensity(elem1)
    cross_section_1 = xraylib.CS_Total(elem1, energy)
    mu_1 = density_1 * cross_section_1 
    
    density_2 = xraylib.ElementDensity(elem2)
    cross_section_2 = xraylib.CS_Total(elem2, energy)
    mu_2 = density_2 * cross_section_2
    
    for x in range(sx):
        for y in range(sx):
            xx = float(x - sx_half)
            yy = float(y - sx_half)
            r = np.sqrt(xx*xx + yy*yy)
            tetta = np.arctan2(yy, xx)

            if r <= sq*(1 + np.cos(2*tetta) + pow(np.sin(2*tetta), 2)) and xx != 0:
                phantom[x, y] = mu_1
            if pow(xx, 2) + pow(yy, 2) <= pow(0.09*sx, 2):
                phantom[x, y] = mu_1
                if (isimplant):
                    if pow(xx / (0.11*sx), 2) + pow(yy / (0.07*sx), 2) < 1 and y <= 128.0 and pow(xx / (0.11*sx), 2) + pow(((yy - 0.025*sx) / (0.07*sx)), 2) > 1:
                        phantom[x, y] = mu_2
            xx = float(x - 5 * sx / 6)
            yy = float(y - sx_half + 1)
            r = np.sqrt(xx*xx + yy*yy)
            tetta = np.arctan2(yy, xx)

            if r <= sq*(1 + np.cos(2*tetta) + pow(np.sin(2*tetta), 2)) and xx != 0:
                phantom[x, y] = mu_1
            if pow(xx, 2) + pow(yy, 2) <= pow(0.09*sx, 2):
                phantom[x, y] = mu_1
            xx = float(x - sx / 6)
            yy = float(y - sx_half - 1)
            r = np.sqrt(xx*xx + yy*yy)
            tetta = np.arctan2(yy, xx)
            if r <= sq * (1 + np.cos(2*tetta) + pow(np.sin(2*tetta), 2)) and xx != 0:
                phantom[x, y] = mu_1
            if pow(xx, 2) + pow(yy, 2) <= pow(0.09*sx, 2):
                phantom[x, y] = mu_1
                
    phantom *= pixel_size
    print("for Ca:")
    print(mu_1*pixel_size)
    print("for Ti")
    print(mu_2*pixel_size)
    return phantom

def create_phantom(size):
    energy = 35.0
    elem1 = 20  # Ca
    elem2 = 81  # Ti
    pixel_size = 0.04
    return create_phantom_tooth(size, energy, elem1, elem2, pixel_size, True)

In [ ]:
size = 512
%timeit original = create_phantom(size)

In [ ]:
plt.imshow(original, cmap=plt.cm.pink)
